In [4]:
import json
import pandas as pd
# import os
# os.getcwd()
with open('songs.json','r', encoding='utf8') as f:
    json_data = json.loads(f.read())

data_df = pd.DataFrame(columns = ['곡명', '가수','앨범', '발매일', '장르','가사'])
data_df.head()
for data in json_data:
    # DataFrame have to assign again
    data_df = data_df.append(pd.Series(data), ignore_index = True)

data_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   곡명      100 non-null    object
 1   가수      100 non-null    object
 2   앨범      100 non-null    object
 3   발매일     100 non-null    object
 4   장르      100 non-null    object
 5   가사      100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB


In [5]:
data_df['가수'].value_counts()

방탄소년단                  8
폴킴                     5
아이유                    5
싹쓰리 (유두래곤, 린다G, 비룡)    3
임영웅                    3
                      ..
산들                     1
조정석                    1
백지영                    1
김필                     1
Red Velvet (레드벨벳)      1
Name: 가수, Length: 71, dtype: int64

In [6]:
data_df.loc[data_df['가수'] == '방탄소년단', ['곡명', '장르','앨범']]

,곡명,장르,앨범
36,작은 것들을 위한 시 (Boy With Luv) (Feat. Halsey),댄스,MAP OF THE SOUL : PERSONA
39,ON,랩/힙합,MAP OF THE SOUL : 7
69,봄날,랩/힙합,YOU NEVER WALK ALONE
74,00:00 (Zero O’Clock),랩/힙합,MAP OF THE SOUL : 7
81,Black Swan,랩/힙합,MAP OF THE SOUL : 7
87,친구,랩/힙합,MAP OF THE SOUL : 7
94,Filter,랩/힙합,MAP OF THE SOUL : 7
95,Moon,랩/힙합,MAP OF THE SOUL : 7


In [7]:
import numpy as np
data_df = data_df.reset_index(drop = True) # I guess this is not necessary
data_df.index = np.arange(1,len(data_df)+1)
data_df.head(3)

,곡명,가수,앨범,발매일,장르,가사
1,다시 여기 바닷가,"싹쓰리 (유두래곤, 린다G, 비룡)",다시 여기 바닷가,2020.07.18,댄스,예아 호우 예예예싹쓰리 인더 하우스커커커커커몬 싹 쓰리 투 렛츠고나 다시 또 설레어...
2,마리아 (Maria),화사 (Hwa Sa),María,2020.06.29,댄스,욕을 하도 먹어 체했어 하도서러워도 어쩌겠어 I do모두들 미워하느라 애썼네날 무너...
3,How You Like That,BLACKPINK,How You Like That,2020.06.26,랩/힙합,보란 듯이 무너졌어바닥을 뚫고 저 지하까지옷 끝자락 잡겠다고저 높이 두 손을 뻗어봐...


In [8]:
# Change column name
data_df.columns = ['name', 'singer', 'album', 'release_date', 'genre', 'lyric']
data_df.head(3)

,name,singer,album,release_date,genre,lyric
1,다시 여기 바닷가,"싹쓰리 (유두래곤, 린다G, 비룡)",다시 여기 바닷가,2020.07.18,댄스,예아 호우 예예예싹쓰리 인더 하우스커커커커커몬 싹 쓰리 투 렛츠고나 다시 또 설레어...
2,마리아 (Maria),화사 (Hwa Sa),María,2020.06.29,댄스,욕을 하도 먹어 체했어 하도서러워도 어쩌겠어 I do모두들 미워하느라 애썼네날 무너...
3,How You Like That,BLACKPINK,How You Like That,2020.06.26,랩/힙합,보란 듯이 무너졌어바닥을 뚫고 저 지하까지옷 끝자락 잡겠다고저 높이 두 손을 뻗어봐...


In [9]:
# Length check for each column

def find_column_max_leng(col):
    return data_df[col].str.len().max()
def find_all(data_frame):
    for column in data_frame.columns:
        print(f'{column} : {find_column_max_leng(column)}')

find_all(data_df)

name : 41
singer : 19
album : 40
release_date : 10
genre : 14
lyric : 2155


* DataFrame to DB Table 

In [10]:

import pymysql
import sqlalchemy
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

engine = create_engine("mysql+mysqldb://python:" + "python" + "@localhost/python_db"\
    , encoding = 'utf-8')
## mysql + mysqldb : ID  + Password + DB name
conn = engine.connect()

data_df.to_sql(name = 'songs', con = engine, if_exists= 'replace', index=True, index_label='id',\
    dtype={
        'id' : sqlalchemy.types.INTEGER(),
        'name' :sqlalchemy.types.VARCHAR(100),
        'singer' :sqlalchemy.types.VARCHAR(100),
        'album' :sqlalchemy.types.VARCHAR(100),
        'release_date' :sqlalchemy.DATE, ## Date type
        'genre' :sqlalchemy.types.VARCHAR(100),
        'lyric' :sqlalchemy.types.VARCHAR(2500)
    })

conn.close()

* DB Table to DataFrame

In [13]:

import pymysql
import sqlalchemy
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

engine = create_engine("mysql+mysqldb://python:" + "python" + "@localhost/python_db"\
    , encoding = 'utf-8')
## mysql + mysqldb : ID  + Password + DB name
conn = engine.connect()
songs_df = pd.read_sql_table('maxpop', conn)
conn.close()

songs_df.head()

,인구수,행정구역,광역시도
0,313851,원주시,강원도
1,1125461,수원시,경기도
2,1010040,창원시,경상남도
3,488395,포항시,경상북도
4,439683,북구,광주광역시


* SQL query to DataFrame

In [18]:

import pymysql
import sqlalchemy
pymysql.install_as_MySQLdb()
from sqlalchemy import create_engine

engine = create_engine("mysql+mysqldb://python:" + "python" + "@localhost/python_db"\
    , encoding = 'utf-8')
## mysql + mysqldb : ID  + Password + DB name
conn = engine.connect()

# select * from songs where album like "%OST %" 
target = 'OST '
sql = """select * from songs where album like %s; """
query_df = pd.read_sql(sql,con = conn, params = ('%' + target + '%', ))
conn.close()

query_df.head()

,id,name,singer,album,release_date,genre,lyric
0,10,아로하,조정석,슬기로운 의사생활 OST Part 3,2020-03-27,"발라드, 국내드라마",어두운 불빛아래 촛불 하나와인 잔에 담긴 약속하나항상 너의 곁에서 널 지켜줄거야날 ...
1,14,사랑하게 될 줄 알았어,전미도,슬기로운 의사생활 OST Part 11,2020-05-22,"발라드, 국내드라마",널 처음 사진으로 본 그날구십구년 일월 삼십일일그날 이후 지금 이 순간까지나 하나만...
2,22,흔들리는 꽃들 속에서 네 샴푸향이 느껴진거야,장범준,멜로가 체질 OST Part 3,2019-08-23,"록/메탈, 국내드라마",흔들리는 꽃들 속에서네 샴푸향이 느껴진거야스쳐지나간건가 뒤돌아보지만그냥 사람들만 보...
3,27,시작,가호 (Gaho),이태원 클라쓰 OST Part.2,2020-02-01,"록/메탈, 국내드라마",새로운 시작은 늘 설레게 하지모든 걸 이겨낼 것처럼시간을 뒤쫓는 시계바늘처럼앞질러 ...
4,29,좋은 사람 있으면 소개시켜줘,조이 (JOY),슬기로운 의사생활 OST Part 2,2020-03-20,"발라드, 국내드라마",좋은 사람 있으면 소개시켜줘때로는 물처럼 때로는 불처럼진심으로 나만을 사랑할 수 있...
